# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter von Domherren einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: 
`domherr.tbl_domherren` und `domherr.tbl_domherren_aemter`

[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Archidiakonat](#Archidiakonat)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt) (hinfällig?)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [10]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [11]:
item_type_id = 5

5

In [12]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-05-18"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-05-18"

In [13]:
out_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

Die Ämter in Tabelle `role` werden im Notebook "Strukturdaten" eingelesen.

## Amtsperiode

In [14]:
using CSV, DataFrames

In [15]:
# Quelle CSV
filename = joinpath(data_path, "tbl_Domherren_Aemter.csv")
df_pr = CSV.read(filename, DataFrame);

In [11]:
# Quelle Datenbank
sql = "SELECT * FROM domherr.tbl_domherren_aemter"
df_pr = Wds.sql_df(sql);

In [16]:
Wds.clean_up!(df_pr);

Ignoriere Einträge ohne Amtsart/ID_Amt_norm

In [17]:
size(df_pr)

(33958, 20)

In [18]:
one_valid(a, b) = !ismissing(a) | !ismissing(b)

one_valid (generic function with 1 method)

In [19]:
subset!(df_pr, [:Amtsart, :ID_Amt_norm] => ByRow(one_valid));

In [20]:
df_pr[101:105, [:ID_Amt, :Amtsart, :ID_Amt_norm, :Amtsbeginn, :Amtsende]]

,ID_Amt,Amtsart,ID_Amt_norm,Amtsbeginn,Amtsende
,Int64,String?,Int64?,String?,String?
1,84465,Domherr,19,1694,1739
2,84466,Dekan,12,1615,missing
3,84467,missing,34,1037/52,missing
4,84468,missing,34,1090,missing
5,84469,missing,34,1096,(1101)


### Domherren zuordnen

In [21]:
transform!(df_pr, :ID_Domherr => ByRow(string) => :id_in_source);

In [22]:
table_name = "item";
sql = "SELECT id AS person_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)"
df_idx = Wds.sql_df(sql);

Lösche Einträge ohne Wert für `ID_Domherr`

In [28]:
dropmissing!(df_pr, :ID_Domherr);

In [29]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

In [30]:
df_idx_mg = filter(:person_id => ismissing, df_prp);

In [31]:
size(df_idx_mg)

(0, 22)

In [32]:
df_idx_mg[:, [:ID_Domherr, :ID_Amt, :Amtsart]]

,ID_Domherr,ID_Amt,Amtsart
,Int64,Int64,String?


### Rollen/Ämter zuordnen

In [33]:
sql = "SELECT name as role_name, role.id as role_id, item.id_in_source as role_id_in_source
FROM role JOIN item ON item.id = role.id"
df_role = Wds.sql_df(sql);

Werden alle Ämter gefunden?

In [34]:
role_dh = unique(string.(df_prp.ID_Amt_norm));

In [35]:
match_mg = setdiff(role_dh, df_role.role_id_in_source)

1-element Vector{String}:
 "missing"

In [36]:
to_string_maybe(a) = ismissing(a) ? a : string(a)

to_string_maybe (generic function with 1 method)

In [37]:
transform!(df_prp, :ID_Amt_norm => ByRow(to_string_maybe) => :match_role_id);

In [43]:
df_prp_r = leftjoin(df_prp, df_role, on = :match_role_id => :role_id_in_source, matchmissing = :notequal);

In [44]:
count(ismissing, df_prp_r.role_id), count(ismissing, df_prp_r.Amtsart)

(1167, 3019)

*2022-03-16* TODO übernimm die Rollenbezeichnung `person_role.role_name` aus der Tabelle `role`.

### Bistümer zuordnen

In [40]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

Es sind nicht alle Ämter mit einem Bistum verknüpft, daher ist eine JOIN-Operation auf diesem Feld nicht möglich.

In [45]:
df_prp_r = leftjoin(df_prp_r, df_dioc, on = :Bistum => :diocese_name, matchmissing = :notequal);

In [46]:
idx_diocese = .!ismissing.(df_prp_r[!, :diocese_id]);

In [47]:
sum(idx_diocese)

926

### Instutionen zuordnen
Klöster, Domstifte

In [48]:
table_name = "institution";
sql = "SELECT id AS institution_id, name, id_gsn FROM $(table_name)"
df_inst = Wds.sql_df(sql);

In [50]:
df_prp_r = leftjoin(df_prp_r, df_inst, on = :ID_Kloster => :id_gsn, matchmissing = :notequal);

Version mit eigener lookup-Funktion

In [43]:
lookup_inst = Dict(df_inst.id_gsn .=> df_inst.institution_id);

In [44]:
length(lookup_inst)

5286

In [45]:
get_inst_id(id_gsn) = get(lookup_inst, id_gsn, missing)

get_inst_id (generic function with 1 method)

In [46]:
get_inst_id(318)

79619

In [47]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_id) => :institution_id);

In [48]:
lookup_inst_name = Dict(df_inst.id_gsn .=> df_inst.name);

In [49]:
get_inst_name(id_gsn) = get(lookup_inst_name, id_gsn, missing)

get_inst_name (generic function with 1 method)

In [50]:
get_inst_name(318)

"Benediktinerinnenkloster Neuenwalde"

In [51]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_name) => :institution_name);

In [51]:
df_mg_inst_id = subset(df_prp_r, :institution_id => ByRow(ismissing));

In [52]:
size(df_mg_inst_id, 1), size(df_prp_r, 1)

(1119, 33756)

### Numerische Daten eintragen

In [53]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
df_prp_r[!, :num_date_begin] .= num_date_begin.(df_prp_r[!, :Amtsbeginn]);

┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1118
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1118
┌ Warning: Could only find year in 
│   s = t 1567
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = Bewerbung 1451
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl

In [54]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper)
df_prp_r[!, :num_date_end] .= num_date_end.(df_prp_r[!, :Amtsende]);

┌ Warning: Could only find year in 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = t1529
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = t1547
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could only find year in 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1113
┌ Warning: Could not parse 
│   s = 66
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1118
┌ Warning: Could not parse 
│   s = 86
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup

In [55]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [56]:
size(df_info, 1)

30737

In [57]:
df_info[60:67, [:ID_Domherr, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

,ID_Domherr,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int64,String,String?,Int64?,String?,Int64?
1,14136,Domkantor,1460,1460,1473,1473
2,14139,Domkantor,1474,1474,1498,1498
3,14141,Domkantor,1510,1510,1517,1517
4,14142,Domkantor,1530,1530,1538,1538
5,14145,Domkantor,1541,1541,1544,1544
6,14148,Domkantor,1553,1553,1572,1572
7,14804,Dompropst,1159?,1159,1174,1174
8,14805,Dompropst,1175,1175,1183,1183


In [58]:
size(df_prp)

(33756, 23)

*2022-05-20* Datenfehler in der Quelle korrigiert

In [94]:
function fix_date(s)
    if ismissing(s)
        return s
    end
    return replace(s, "1 Hälfte" => "1. Hälfte", "14 Jahr" => "14. Jahr", "Beginn16" => "Beginn 16")
end

fix_date (generic function with 1 method)

In [98]:
transform!(df_prp_r, :Amtsbeginn => ByRow(fix_date) => :Amtsbeginn);

In [99]:
transform!(df_prp_r, :Amtsende => ByRow(fix_date) => :Amtsende);

In [107]:
transform!(df_prp_r, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = (1553) 1560
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = (1574) 1593
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = t 1567
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = (Bewerbung 1451)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warni

*2022-05-12* In `tbl_Domherren_Aemter` enthalten 200 Einträge ein `†`. Das wird vor dem Parsen entfernt.

In [108]:
transform!(df_prp_r, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: could not parse 
│   s = (1101)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: year out of range in 
│   s = 16445
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1530
┌ Warning: could not parse 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = t1529
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = t1547
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could not parse 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1538
┌ Warning: could no

In [105]:
maximum(df_prp_r.date_sort_key)

9000900

In [109]:
replace_if_missing(a, b) = a == 9000900 ? b : a

replace_if_missing (generic function with 1 method)

In [110]:
transform!(df_prp_r, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

In [111]:
size(df_prp_r, 1), count(isequal(9000900), df_prp_r.date_sort_key)

(33756, 1845)

In [112]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [114]:
df_info[100:105, [:Amtsart, :ID_Amt_norm, :role_id, :Amtsbeginn, :num_date_begin, :num_date_end, :date_sort_key]]

,Amtsart,ID_Amt_norm,role_id,Amtsbeginn,num_date_begin,num_date_end,date_sort_key
,String,Int64?,Int32?,String?,Int64?,Int64?,Int64
1,Domscholaster,36,249163,vor 1328,1278,missing,1328100
2,Domkantor,26,249113,1356,1356,1374,1356150
3,Domkantor,26,249113,1376,1376,1376,1376150
4,Domkantor,26,249113,1405,1405,1419,1405150
5,Domkantor,26,249113,1421,1421,1447,1421150
6,Domkantor,26,249113,1452,1452,1455,1452150


Einträge für diesen Item_Typ löschen

In [115]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_id))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 33756, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer - auch für die identifizierten - den Namen des Bistums in die Tabelle.  
Schreibe für alle Institutionen - auch für die identifizieren - die Bezeichnung in die Tabelle.  
Schreibe für alle Ämter - auch für die identifizieren - die Bezeichnung in die Tabelle.

Erzeuge die IDs selbst, um sie für `person_role_property` zur Verfügung zu haben.

In [116]:
table_name = "person_role"
sql = "SELECT MAX(id) + 1 AS next_id FROM $(table_name)"
df_next_id = Wds.sql_df(sql)

,next_id
,Int64?
1,516558


In [117]:
next_id = df_next_id[1, :next_id] + 1

516559

In [118]:
df_prp_r[!, :id] .= collect(next_id:(next_id + size(df_prp_r, 1) - 1));

In [119]:
df_prp_r[1001:1007, [:id, :person_id, :ID_Domherr, :Amtsart, :ID_Amt]]

,id,person_id,ID_Domherr,Amtsart,ID_Amt
,Int64,Int32?,Int64,String?,Int64
1,517559,292130,45146,missing,82350
2,517560,292129,45147,missing,82351
3,517561,292129,45147,missing,82352
4,517562,292129,45147,missing,82353
5,517563,292129,45147,missing,82354
6,517564,292129,45147,missing,82355
7,517565,292081,45135,missing,82356


In [120]:
columns = [
    :id => :id,
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :Institution => :institution_name,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [121]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prp_r, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: 30000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: Rows inserted: 33756
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


33756

Titularbistum ist noch nicht eingetragen

## Zeitspanne der Person

In [122]:
table_name = "person"
sql = "SELECT id, num_date_birth, num_date_death
FROM $(table_name) WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [123]:
size(df_date)

(22938, 3)

Zeitspanne der Ämter

In [124]:
df_pr_begin_0 = dropmissing(df_prp_r, :num_date_begin)
df_pr_begin = groupby(select(df_pr_begin_0, :person_id, :num_date_begin), :person_id)

,person_id,num_date_begin
,Int32?,Int64
1,269991,1680
2,269991,1678
3,269991,1677
,person_id,num_date_begin
,Int32?,Int64
1,292980,1112


In [125]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [126]:
size(df_pr_date_min)

(22337, 2)

In [127]:
df_pr_date_min[3015:3021, :]

,person_id,date_min
,Int32?,Int64
1,273030,1435
2,273031,1437
3,273032,1435
4,273033,1439
5,273034,1436
6,273035,1428
7,273036,1387


`date_max`

In [128]:
df_pr_end_value = dropmissing(df_prp_r, :num_date_end)
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id)

,person_id,num_date_end
,Int32?,Int64
1,269991,1690
2,269991,1690
3,269991,1690
,person_id,num_date_end
,Int32?,Int64
1,292980,1122


In [129]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [130]:
size(df_pr_date_max)

(16184, 2)

In [131]:
df_pr_date_max[3015:3021, :]

,person_id,date_max
,Int32?,Int64
1,273936,1422
2,273937,1411
3,273938,1382
4,273939,1349
5,273941,1344
6,273942,1336
7,273943,1325


In [132]:
df_pr_date_min_mng = subset(df_pr_date_min, :person_id => ByRow(ismissing));

In [133]:
size(df_pr_date_min_mng)

(0, 2)

In [134]:
subset!(df_pr_date_min, :person_id => ByRow(!ismissing));

In [135]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [136]:
subset!(df_pr_date_max, :person_id => ByRow(!ismissing));

In [137]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [138]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [139]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [140]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [141]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [142]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [143]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [144]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein

In [145]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [146]:
filename = joinpath(out_path, "cn_date_person_update.sql");
table_name = "person";
Wds.update_sql(filename, table_name, select(df_date, columns), on = :id);

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 12000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 14000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 16000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1628
┌ Info: row 
│   i = 18000
└

Alternativ über temporäre Tabelle

In [100]:
sql = "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) \nENGINE=InnoDB DEFAULT CHARSET=utf8;", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [101]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [102]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [103]:
table_name = "date_update"
Wds.filltable!(table_name, select(df_date, columns));

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: Rows inserted: 22278
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


In [104]:
sql = "UPDATE person AS p, 
(SELECT id, date_min, date_max FROM date_update) AS du
SET p.date_min = du.date_min, p.date_max = du.date_max
WHERE p.id = du.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "UPDATE person AS p, \n(SELECT id, date_min, date_max FROM date_update) AS du\nSET p.date_min = du.date_min, p.date_max = du.date_max\nWHERE p.id = du.id", 0, -1, 22278, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

*2022-01-17* Es gibt etliche Einträge für Personen mit Ämtern nach 2000?! 
```sql
select p.id, id_in_source, item_type_id, givenname, familyname, date_min, date_max, edit_status, is_online
from person as p join item on item.id = p.id and p.date_max > 2000;
```

## Archidiakonat

In [72]:
println.(names(df_prp_r));

Bistum
Amtsart
Amtsbeginn
Amtsende
ID_Amt
Institution
Amt_weltlich
Herrschaftsgebiet
Ort
Amt_Kommentar
Sort
Titularbistum
Profession
ID_Kloster
ID_Fremdschlüssel
Dignität
ID_Domherr
Archidiakonat
Hilfsfeld_import
ID_Amt_norm
id_in_source
person_id
match_role_id
role_name
role_id
diocese_id
institution_id
institution_name
num_date_begin
num_date_end
date_sort_key
date_sort_key_replace
id


In [147]:
columns = [
    :id => :person_role_id,
    :person_id => :person_id,
    :Archidiakonat => :value
]

3-element Vector{Pair{Symbol, Symbol}}:
            :id => :person_role_id
     :person_id => :person_id
 :Archidiakonat => :value

In [148]:
df_pr_property = select(df_prp_r, columns);

In [149]:
dropmissing!(df_pr_property, :value);

In [150]:
size(df_pr_property)

(511, 3)

In [151]:
df_pr_property[!, :name] .= "Archidiakonat";

In [152]:
df_pr_property[407:414, :]

,person_role_id,person_id,value,name
,Int64,Int32?,String,String
1,549298,278272,Plau,Archidiakonat
2,549299,278253,Rostock,Archidiakonat
3,549300,278324,Parchim,Archidiakonat
4,549301,278320,Tribsees,Archidiakonat
5,549302,278317,Rostock,Archidiakonat
6,549303,278313,Waren,Archidiakonat
7,549304,278303,Rostock,Archidiakonat
8,549305,278302,Tribsees,Archidiakonat


In [153]:
table_name = "person_role_property";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role_property\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [154]:
table_name = "person_role_property"
Wds.filltable!(table_name, df_pr_property)

┌ Info: Rows inserted: 511
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


511

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-17* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12